ref:https://github.com/booknlp/booknlp

In [1]:
!pip3 install torch torchvision

In [2]:
import torch
x = torch.rand(5, 3)
print(x)

tensor([[0.4434, 0.4965, 0.7209],
        [0.1129, 0.4342, 0.9875],
        [0.8240, 0.9767, 0.1579],
        [0.9919, 0.9404, 0.0292],
        [0.4194, 0.9506, 0.3798]])


In [3]:
!conda create --name booknlp python=3.7
!conda activate booknlp

Error while loading conda entry point: anaconda-cloud-auth (cannot import name 'ChannelAuthBase' from 'conda.plugins.types' (/Users/meinv/anaconda3/lib/python3.11/site-packages/conda/plugins/types.py))
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): repo.anaconda.com:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): repo.anaconda.com:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): repo.anaconda.com:443
/ DEBUG:urllib3.connectionpool:https://repo.anaconda.com:443 "GET /pkgs/r/osx-arm64/current_repodata.json HTTP/1.1" 304 0
DEBUG:urllib3.connectionpool:https://repo.anaconda.com:443 "GET /pkgs/main/noarch/current_repodata.json HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://repo.anaconda.com:443 "GET /pkgs/main/osx-arm64/current_repodata.json HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://repo.anaconda.com:443 "GET /pkgs/r/noarch/current_repodata.json HTTP/1.1" 304 0
done
Solving environment: unsuccessful attempt using

In [4]:
!pip install booknlp
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 27.1 MB/s eta 0:00:0000:0100:01


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [5]:
import json
from collections import Counter

In [6]:
def proc(filename):
    with open(filename) as file:
        data=json.load(file)
    return data

In [7]:
data=proc("output_Ao3_10/op_An_Unlikely_Comfort/001.book")

In [8]:
def get_counter_from_dependency_list(dep_list):
    counter=Counter()
    for token in dep_list:
        term=token["w"]
        tokenGlobalIndex=token["i"]
        counter[term]+=1
    return counter

In [9]:
for character in data["characters"]:
    possCounter=Counter()
    
    agentList=character["agent"]
    patientList=character["patient"]
    possList=character["poss"]
    modList=character["mod"]
    
    character_id=character["id"]
    count=character["count"]

    referential_gender_distribution=referential_gender_prediction="unknown"

    if character["g"] is not None and character["g"] != "unknown":
        referential_gender_distribution=character["g"]["inference"]
        referential_gender=character["g"]["argmax"]
    
    mentions=character["mentions"]
    proper_mentions=mentions["proper"]
    max_proper_mention=""
    
    # print out the characters with proper names, along with their syntactic information
    if len(mentions["proper"]) > 0:
        max_proper_mention=mentions["proper"][0]["n"]
    
        print(character_id, count, max_proper_mention, referential_gender)

        print()
        printTop=10
        for k, v in get_counter_from_dependency_list(possList).most_common(printTop):
            print("\tposs\t%s %s" % (v,k))
        print()
        for k, v in get_counter_from_dependency_list(agentList).most_common(printTop):
            print("\tagent\t%s %s" % (v,k))        
        print()
        for k, v in get_counter_from_dependency_list(patientList).most_common(printTop):
            print("\tpatient\t%s %s" % (v,k))       
        print()
        for k, v in get_counter_from_dependency_list(modList).most_common(printTop):
            print("\tmod\t%s %s" % (v,k))    
        print()

6 64 Riff Raff he/him/his

	poss	2 company
	poss	1 songs
	poss	1 toil
	poss	1 planet
	poss	1 way
	poss	1 quarters
	poss	1 master
	poss	1 gloves
	poss	1 brain
	poss	1 sister

	agent	2 feel
	agent	2 noticed
	agent	1 pushes
	agent	1 care
	agent	1 like
	agent	1 enjoyed
	agent	1 did
	agent	1 felt
	agent	1 making
	agent	1 stopped

	patient	2 noticed
	patient	1 mocking
	patient	1 annoy
	patient	1 hug
	patient	1 taken
	patient	1 Thank

	mod	1 confused
	mod	1 figure

8 15 Eddie he/him/his

	poss	1 sister
	poss	1 Magenta
	poss	1 master
	poss	1 mind

	agent	1 tried
	agent	1 did

	patient	1 killed
	patient	1 kill
	patient	1 wake
	patient	1 killing
	patient	1 loved
	patient	1 dismembering


7 11 Columbia she/her


	agent	1 started
	agent	1 expected

	patient	1 pushes
	patient	1 saw
	patient	1 save


10 6 Frank he/him/his


	agent	1 made
	agent	1 attempting
	agent	1 dismembering

	patient	1 stop
	patient	1 loved


12 3 Magenta he/him/his


	agent	1 seem
	agent	1 hug


	mod	1 better

14 3 Both they/t

In [15]:
import re
# Because we're going to use the split() function in this module.
## https://docs.python.org/3/library/re.html

#I want to see how many sentences there are in this article.
def read_text(filename):
    with open(filename, 'r', encoding='utf-8') as file:
        text = file.read()
    return text
# 
def count_sentences(text):
    sentences = re.split(r'\.', text)  
    return len(sentences)


filename = "RH_10_txt/txt_exports/An_Unlikely_Comfort.txt"
text = read_text(filename)


num_sentences = count_sentences(text)
print("Total number of sentences:", num_sentences)




Total number of sentences: 51


In [12]:
import pandas as pd

#Create an empty DataFrame to store the number of co-occurrences

def count_cooccurrences(text, names, window_size):
    cooccurrence_counts = pd.DataFrame(index=names, columns=names).fillna(0)  
    
    # Split text into sentences based on English periods
    sentences = re.split(r'\.', text)  
    for sentence in sentences:
        for i in range(len(names)):
            for j in range(i + 1, len(names)):
                name1 = names[i]
                name2 = names[j]
                if name1 in sentence and name2 in sentence:
                    # Count the number of times two characters appear in the same sentence, 
                    # and the co-occurrence of the two characters is within the restricted window length.
                    name1_index = sentence.find(name1)
                    name2_index = sentence.find(name2)
                    
                    #Make sure the difference in index is an absolute value.
                    #Because it is an undirected graph, both directions must be added once.
                    if abs(name1_index - name2_index) <= window_size:
                        cooccurrence_counts.at[name1, name2] += 1
                        cooccurrence_counts.at[name2, name1] += 1  

    return cooccurrence_counts

# Define character name and window size.
names = ["Riff Raff", "Eddie", "Columbia", "Frank", "Magenta", "Rocky"]
window_size = len(text) // 4 

cooccurrence_counts = count_cooccurrences(text, names, window_size)

print(cooccurrence_counts)


           Riff Raff  Eddie  Columbia  Frank  Magenta  Rocky
Riff Raff          0      3         4      0        0      2
Eddie              3      0         3      3        0      3
Columbia           4      3         0      1        2      2
Frank              0      3         1      0        0      1
Magenta            0      0         2      0        0      0
Rocky              2      3         2      1        0      0
